### Data Quality Check for FACT FINANCIALS


This notebook provides a review of data quality, structure, and integrity check for the table fact_financials before jumping to the power bi visualization. 

### Database configuration

This notebook reads database credentials from environment variables.
Credentials are **not stored in the notebook or repository**.

Required variables:

- DB_USER
- DB_PASSWORD
- DB_HOST
- DB_NAME

For local development:

- Create a `.env` file (see `.env.example`)
- Load variables using `python-dotenv`

### 1. Import Necessary Library

In [1]:
# Core libraries for data manipulation and numerical operations
import numpy as np
import pandas as pd

#import sql engine
from sqlalchemy import create_engine 

### 2. Load Data

In [2]:
#import dataset from mysql
from dotenv import load_dotenv
import os

load_dotenv()

engine = create_engine(
    f"mysql+mysqlconnector://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}"
    f"@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)

query="SELECT * FROM gdb041.fact_financials"
df_fact=pd.read_sql(query, engine)
df_fact.head()

,date,fiscal_year,customer_code,product_code,market,sold_quantity,gross_sales,net_invoice_sales,net_sales,manufacturing_cost,freight_cost,COGS,gross_margin,op_expense,net_profit
0,2017-09-01,2018,70002017,A0118150101,India,51,785.1552,720.4253,477.0458,235.5690,12.8802,248.4492,228.5966,214.8137,13.7828
1,2017-09-01,2018,70002017,A0118150102,India,77,1508.2375,1383.8951,945.0524,431.4772,25.5164,456.9936,488.0588,425.5571,62.5017
2,2017-09-01,2018,70002017,A0118150103,India,54,1045.6020,959.4003,583.0104,321.1326,15.7413,336.8739,246.1365,262.5296,-16.3931
3,2017-09-01,2018,70002017,A0118150104,India,42,822.1206,754.3432,498.5636,247.6236,13.4612,261.0848,237.4788,224.5032,12.9756
4,2017-09-01,2018,70002017,A0418150101,India,67,1063.6451,975.9559,588.8888,319.6771,15.9000,335.5771,253.3117,265.1766,-11.8649


### 3. Initial Data Quality Check

In [3]:
def initial_report(df):
    print(" *** initial report ***\n" + "-"*40)

    print(f"*** Structure:\n- Total Rows: {df.shape[0]}\n- Total Columns: {df.shape[1]}")
    print(f"- Column Names: {list(df.columns)}\n")

    
    print(" Data Types:")
    for col, dtype in df.dtypes.items():
        print(f"  {col}: {dtype}")
    print()

    print("Mixed Data Types:")
    has_mixed_types = False
    for col in df.columns:
        try:
            type_counts = df[col].apply(type).value_counts()
            if len(type_counts) > 1:
                has_mixed_types = True
                print(f"  {col}:")
                for t, count in type_counts.items():
                    print(f"    - {t.__name__}: {count}")
        except Exception as e:
            print(f"  {col}: Error checking types - {e}")

    if not has_mixed_types:
        print("  No mixed data types found")
    print()

    print("*** Distinct Values per Column:")
    for col in df.columns:
        print(f"  {col}: {df[col].nunique()}")
    print()

    print("*** Null Values and Percentages:")
    has_null_value=False
    nulls = df.isnull().sum()
    for col in df.columns:
        pct_missing = np.mean(df[col].isnull())
        if nulls[col] > 0: # Only print if there are missing values
            has_null_value=True
            print(f"  {col}: Missing Values: {nulls[col]}, Pct: {round(pct_missing * 100, 3)}%")
    if not has_null_value:
        print("  No null values found")
    print()

    
    print(f"\n*** Duplicates: {df.duplicated().sum()}")

    print("*** Negative or Zero Values:")
    has_issues = False
    for col in df.select_dtypes(include='number').columns:
        zero_count = (df[col] == 0).sum()
        negative_count = (df[col] < 0).sum()
    
        if zero_count > 0 or negative_count > 0:
            has_issues = True
            print(f"  {col}:")
            if zero_count > 0:
                print(f"    - Zero values: {zero_count}")
            if negative_count > 0:
                print(f"    - Negative values: {negative_count}")

    if not has_issues:
        print("  No negative or zero values found")
    print()

initial_report(df_fact)

 *** initial report ***
----------------------------------------
*** Structure:
- Total Rows: 1424923
- Total Columns: 15
- Column Names: ['date', 'fiscal_year', 'customer_code', 'product_code', 'market', 'sold_quantity', 'gross_sales', 'net_invoice_sales', 'net_sales', 'manufacturing_cost', 'freight_cost', 'COGS', 'gross_margin', 'op_expense', 'net_profit']

 Data Types:
  date: object
  fiscal_year: int64
  customer_code: object
  product_code: object
  market: object
  sold_quantity: int64
  gross_sales: float64
  net_invoice_sales: float64
  net_sales: float64
  manufacturing_cost: float64
  freight_cost: float64
  COGS: float64
  gross_margin: float64
  op_expense: float64
  net_profit: float64

Mixed Data Types:
  No mixed data types found

*** Distinct Values per Column:
  date: 52
  fiscal_year: 5
  customer_code: 209
  product_code: 389
  market: 27
  sold_quantity: 2990
  gross_sales: 256087
  net_invoice_sales: 1122737
  net_sales: 1364251
  manufacturing_cost: 216698
  frei

### Notes:
- Date is object, but in sql it is date.
- fiscal_year, market has low cardinality. They can be used as a slicer

### 3.1: Check Negative values in gross_margin

In [4]:
df_fact[df_fact.gross_margin<0]

,date,fiscal_year,customer_code,product_code,market,sold_quantity,gross_sales,net_invoice_sales,net_sales,manufacturing_cost,freight_cost,COGS,gross_margin,op_expense,net_profit
727893,2020-11-01,2021,90012035,A4419110407,Germany,8,2299.9472,1599.6288,735.8394,717.8544,21.0450,738.8994,-3.0600,254.8948,-257.9548
982342,2021-04-01,2021,90012035,A6720160103,Germany,40,177.6440,123.5526,56.7593,55.1720,1.6233,56.7953,-0.0360,19.6614,-19.6974
1183960,2021-08-01,2021,90012035,A4419110408,Germany,4,1188.1108,826.3391,375.2128,367.2032,10.7311,377.9343,-2.7215,129.9737,-132.6952


### Notes:
COGS is higher than net_sales

### 3.2: Check Negative values in net_profit

In [5]:
df_fact[df_fact.net_profit<0].head()

,date,fiscal_year,customer_code,product_code,market,sold_quantity,gross_sales,net_invoice_sales,net_sales,manufacturing_cost,freight_cost,COGS,gross_margin,op_expense,net_profit
2,2017-09-01,2018,70002017,A0118150103,India,54,1045.6020,959.4003,583.0104,321.1326,15.7413,336.8739,246.1365,262.5296,-16.3931
4,2017-09-01,2018,70002017,A0418150101,India,67,1063.6451,975.9559,588.8888,319.6771,15.9000,335.5771,253.3117,265.1766,-11.8649
9,2017-09-01,2018,70002017,A1018150101,India,54,5228.1936,4797.1698,2853.2159,1531.8126,77.0368,1608.8494,1244.3665,1284.8031,-40.4366
11,2017-09-01,2018,70002017,A1018150103,India,15,1577.4915,1447.4396,895.4634,491.6520,24.1775,515.8295,379.6339,403.2272,-23.5933
12,2017-09-01,2018,70002017,A1118150201,India,47,5259.4833,4825.8799,2907.4764,1563.5302,78.5019,1642.0321,1265.4443,1309.2366,-43.7923


### Notes:
- Operational expense is higher than gross_margin

### 4. Check what markets are available in the datasets

In [6]:
# Display value counts for each column to validate uniqueness and detect potential data quality issues.
df_fact["market"].value_counts(dropna=False).sort_index()

market
Australia          55612
Austria            34938
Bangladesh         33484
Brazil             14021
Canada             84873
Chile               8755
China              23031
Columbia            3782
France             77365
Germany            76076
India             147278
Indonesia          31996
Italy              80172
Japan              52477
Mexico             12736
Netherlands        56532
Newzealand         44815
Norway             50343
Pakistan           34374
Philiphines        47798
Poland             53018
Portugal           77571
South Korea        40922
Spain              59350
Sweden             25174
USA               121400
United Kingdom     77030
Name: count, dtype: int64

### 5. Check basic statistics for sold_quantity
sold_quantity is chosen because all other columns are derived based on that.

In [7]:
df_fact["sold_quantity"].describe()

count    1.424923e+06
mean     8.792865e+01
std      2.080726e+02
min      1.000000e+00
25%      4.000000e+00
50%      1.700000e+01
75%      7.700000e+01
max      5.832000e+03
Name: sold_quantity, dtype: float64

### Notes:
- Total Count: 1424923
- Mean: 87.9
- Std: 208.1
- Median: 17
- min: 1
- max: 5832

As median<mean, the data is right skewed. A very few market/customer/product is driving more sell than others. Data must have outliers.

### 6. Check Outliers

In [8]:
def outliers_iqr(df, columns=None):
    """
    Detect outliers in a DataFrame using the Interquartile Range (IQR) method.
    
    Returns: outlier information for each column
    """
    import pandas as pd
    
    # Select columns to analyze
    if columns is None:
        columns = df.select_dtypes(include='number').columns.tolist()
    
    print("*** Outliers (IQR method):")
    
    for col in columns:
        # Calculate IQR bounds
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Find outliers
        outlier_mask = (df[col] < lower_bound) | (df[col] > upper_bound)
        outliers = df[outlier_mask]
        outlier_count = len(outliers)
        
        # Print summary
        if outlier_count > 0:
            print(f"  {col}: {outlier_count} outliers")
        else:
            print(f"  {col}: No outliers")
        print(f"    Lower Bound: {lower_bound:.2f}")
        print(f"    Upper Bound: {upper_bound:.2f}")
    


# usage:
outliers_iqr(df_fact, columns=['sold_quantity'])

*** Outliers (IQR method):
  sold_quantity: 172069 outliers
    Lower Bound: -105.50
    Upper Bound: 186.50


### 7. Get totals 
sold_quantity, gross_sales, net_invoice_sales, net_sales, manufacturing_cost, frieght_cost, cogs, gross_margin,operational_expense, net_profit

In [9]:
for col in df_fact.select_dtypes(include='number'):
    print(col, df_fact[col].sum())

fiscal_year 2879095716
sold_quantity 125291556
gross_sales 5726646369.820101
net_invoice_sales 4383891667.674201
net_sales 2875220015.412701
manufacturing_cost 1709555909.9243996
freight_cost 88690134.67330001
COGS 1798246044.5977
gross_margin 1076973970.8765
op_expense 1364909515.349
net_profit -287935544.7226


### Notes:
- Numbers are verified with SQL

### 8. Check duplicates in grain level

In [11]:
dupe_keys = (
    df_fact
    .groupby(['date', 'product_code', 'customer_code'])
    .size()
    .reset_index(name='count')
    .query('count > 1')
)
dupe_keys

,date,product_code,customer_code,count


### Notes:
No duplicates at grain level